<a href="https://colab.research.google.com/github/paulo037/text2sqsl/blob/main/preprocessing/EN2PT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q -U datasets huggingface-hub transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 50.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.2 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from google.colab import userdata
import datasets

In [ ]:
from google.colab import userdata
READ_TOKEN = userdata.get('hf_read')
WRITE_TOKEN = userdata.get('hf_write')

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("unicamp-dl/translation-en-pt-t5",
                                              device_map="auto",
                                              use_cache=True)

model = AutoModelForSeq2SeqLM.from_pretrained("unicamp-dl/translation-en-pt-t5",
                                              trust_remote_code=True,
                                              device_map="auto",
                                              use_cache=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.95k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/756k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.35M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/634 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

In [ ]:
dataset = load_dataset("NESPED-GEN/spider", token=READ_TOKEN )

Generating train split:   0%|          | 0/7944 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1867 [00:00<?, ? examples/s]

In [ ]:
pipe = pipeline('text2text-generation', model=model, tokenizer=tokenizer, max_new_tokens=200)

In [ ]:
pipe("translate English to Portuguese: Select the name and price of all products with a price larger than or equal to $180, and sort first by price (in descending order), and then by name (in ascending order).")

[{'generated_text': 'Seleccionar o nome e o preço de todos os produtos com um preço igual ou superior a 180 e ordenar primeiro pelo preço (em ordem decrescente) e depois pelo nome (em ordem decrescente).'}]

In [ ]:
def add_pt_question(line):
  out = pipe(f"translate English to Portuguese: {line['question']}")
  line['question_pt'] = out[0]['generated_text']
  line['question_en'] = line['question']
  return line
dataset = dataset.map(add_pt_question, remove_columns=['question'])  # new column with 6 elements: [0, 1, 2, 0, 1, 2]

In [ ]:
dataset.push_to_hub("NESPED-GEN/spider", token=WRITE_TOKEN)